In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
#importando datasets http://www.respeitoavida.sp.gov.br/relatorios/
df_obitos_raw=pd.read_csv(r'C:\Users\Renato\OneDrive\Projeto_TERA\Predição Acidentes Motocicleta\Datasets\obitos_publico.csv',encoding='unicode_escape',sep=';')
df_nao_fatais_raw = pd.read_csv(r'C:\Users\Renato\OneDrive\Projeto_TERA\Predição Acidentes Motocicleta\Datasets\acidentes_naofatais.csv',encoding='unicode_escape',sep=';')
df_fatais_raw = pd.read_excel(r'C:\Users\Renato\OneDrive\Projeto_TERA\Predição Acidentes Motocicleta\Datasets\acidentes_fatais.xlsx')

C:\Users\Renato\anaconda3\envs\Data_Science\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
#checando duplicatas
df_obitos_raw.duplicated().value_counts() #achado 97 linhas duplicadas (os outros df's não tem duplicatas)
df_obitos_raw = df_obitos_raw.drop_duplicates()#dropando as 97 duplicatas, temos 34657 linhas

In [4]:
# Criando Dataframes para trabalhar
df_nao_fatais = df_nao_fatais_raw.copy()

df_fatais = df_fatais_raw.copy()#NAO TEMOS COMO FILTRAR POR MOTOCILCETA POIS NAO TEM ESSA COLUNA!!!!

df_obitos = df_obitos_raw.copy()#AQUI CONSEGUIMOS FILTRAR POR MOTOCICLETA

In [5]:
print('obitos',df_obitos.shape)
print('fatais',df_fatais.shape)
print('Ñ fatais',df_nao_fatais.shape)

obitos (34657, 34)
fatais (31791, 32)
Ñ fatais (377613, 55)


In [6]:
#Arrumando o nome das colunas
df_obitos = df_obitos_raw.rename(columns={'LAT_(GEO)':'Latitude',
                                          'LONG_(GEO)':'Longitude',
                                          'Tipo de via':'Tipo de Via',
                                         'Id da Delegacia (RDO)':'Id Delegacia', 'Número do Bo (RDO)':'Número BO',
                                          'Ano do BO (RDO)':'Ano BO','Númeral / KM':'Numero/KM'})

df_nao_fatais = df_nao_fatais_raw.rename(columns={'LAT_(GEO)':'Latitude','LONG_(GEO)':'Longitude'})


df_fatais =df_fatais_raw.rename(columns={'Lat (GEO)':'Latitude','Long (GEO)':'Longitude','Numeral / KM':'Numero/KM',
                                         'Iluminação da via (SIOPM)':'Iluminação',
                                        'Superfície da Via (SIOPM)':'Superfície da via', 
                                         'Dia da semana':'Dia da Semana',
                                         'Tipo de via':'Tipo de Via',
                                         'Tipo de pista (SIOPM)':'Tipo de pista',
                                         'Id Delegacia (RDO)':'Id Delegacia', 'Número BO (RDO)':'Número BO', 
                                         'Ano BO (RDO)':'Ano BO',
                                        'Condições Climáticas (SIOPM)':'Condições Climática'})

In [7]:
#alterando tipos de variáveis
df_obitos['Data do Acidente'] = pd.to_datetime(df_obitos['Data do Acidente'],errors='coerce')

df_nao_fatais['Data do Acidente'] = pd.to_datetime(df_nao_fatais['Data do Acidente'],format='%Y-%m-%d')#,errors='coerce'
df_nao_fatais['Ano do Acidente'] = pd.to_datetime(df_nao_fatais['Ano do Acidente'],format='%Y')
df_nao_fatais['Dia do Acidente'] = pd.to_datetime(df_nao_fatais['Dia do Acidente'],format='%d')

df_fatais.loc[31790,'Data do Acidente']='2020-02-27'
df_fatais['Data do Acidente'] = pd.to_datetime(df_fatais['Data do Acidente'],format='%Y-%m-%d')#,errors='coerce'
df_fatais.loc[31790, 'Ano do Acidente']='2020'
df_fatais['Ano do Acidente'] = pd.to_datetime(df_fatais['Ano do Acidente'],format='%Y')
df_fatais['Dia do Acidente'] = pd.to_datetime(df_fatais['Dia do Acidente'],format='%d')

In [8]:
#juntando a df_fatais_geral com df_nao_fatais criando df_acidentes
df_fatais_geral = df_fatais.merge(df_obitos,
                                  on=['Número BO','Data do Acidente','Ano BO','Hora do Acidente','Turno','Município','Região Administrativa','Jurisdição','Administração','Conservação','Outro Veículo Envolvido','Tipo de Via','Logradouro','Dia da Semana','Numero/KM','Id Delegacia','Latitude','Longitude'])

In [9]:
print('obitos',df_obitos.shape)
print('fatais',df_fatais.shape)
print('Ñ fatais',df_nao_fatais.shape)
print('merge',df_fatais_geral.shape)

obitos (34657, 34)
fatais (31791, 32)
Ñ fatais (377613, 55)
merge (33934, 48)


In [18]:
#criando colunas com labels de fatal e não falal
df_fatais_geral['Acidente Fatal']=1
df_fatais_geral['Acidente Não Fatal']=0
df_nao_fatais['Acidente Fatal']=0
df_nao_fatais['Acidente Não Fatal']=1

In [19]:
#juntando a df_fatais_geral com df_nao_fatais criando df_acidentes
df_acidentes = pd.concat([df_nao_fatais,df_fatais_geral])

In [20]:
print('df_obitos',df_obitos.shape)
print('df_fatais',df_fatais.shape)
print('df_nao_fatais',df_nao_fatais.shape)
print('df_fatais_geral',df_fatais_geral.shape)
print('df_acidentes',df_acidentes.shape)

df_obitos (34657, 34)
df_fatais (31791, 32)
df_nao_fatais (377613, 57)
df_fatais_geral (33934, 50)
df_acidentes (411547, 82)


In [21]:
df_acidentes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 411547 entries, 0 to 33933
Data columns (total 82 columns):
 #   Column                                       Non-Null Count   Dtype         
---  ------                                       --------------   -----         
 0   ID                                           411547 non-null  int64         
 1   Data do Acidente                             411547 non-null  datetime64[ns]
 2   Dia do Acidente                              411547 non-null  datetime64[ns]
 3   Mês do Acidente                              411547 non-null  object        
 4   Ano do Acidente                              411547 non-null  datetime64[ns]
 5   Ano/Mês do Acidente                          411547 non-null  float64       
 6   Dia da Semana                                411547 non-null  object        
 7   Hora do Acidente                             411547 non-null  object        
 8   Turno                                        411547 non-null  obj

In [16]:
df_acidentes['Condições da pista'].value_counts()

NAO DISPONIVEL    162240
DUPLA              99503
SIMPLES            92544
MÚLTIPLA           22388
MULTIPLA             936
INCLINADO              1
PLANO                  1
Name: Condições da pista, dtype: int64

In [17]:
df_acidentes['Tipo de pista'].value_counts()

NAO DISPONIVEL    191080
DUPLA             101932
SIMPLES            94776
MULTIPLA           23757
INCLINADO              1
PLANO                  1
Name: Tipo de pista, dtype: int64

In [ ]:
#PRECISAMOS CHECAR COLUNAS ANTES DO REPLACE NAN

In [12]:
#Substituir os nao disponiveis por nulos
# df_acidentes.replace('NAO DISPONIVEL', np.nan,inplace=True)

In [28]:
# Filtros
df_acidentes_moto = df_acidentes.loc[df_acidentes['Veículos Envolvidos - Motocicleta']==1] #
df_acidentes_moto_sp = df_acidentes_moto.loc[df_acidentes_moto['Região Administrativa']=='Metropolitana de São Paulo']# dos 153k, 54692 é SP
# df_nao_fatais_sp_moto = df_nao_fatais_sp_moto.loc[df_nao_fatais_sp_moto['Data do Acidente']>='2019-01-01']
# df_nao_fatais_sp_moto = df_nao_fatais_sp_moto.loc[df_nao_fatais_sp_moto['Data do Acidente']<='2020-12-31']

In [29]:
print('df_obitos',df_obitos.shape)
print('df_fatais',df_fatais.shape)
print('df_nao_fatais',df_nao_fatais.shape)
print('df_fatais_geral',df_fatais_geral.shape)
print('df_acidentes',df_acidentes.shape)
print('df_acidentes_moto',df_acidentes_moto.shape)
print('df_acidentes_moto_sp',df_acidentes_moto_sp.shape)


df_obitos (34657, 34)
df_fatais (31791, 32)
df_nao_fatais (377613, 57)
df_fatais_geral (33934, 50)
df_acidentes (411547, 82)
df_acidentes_moto (168023, 82)
df_acidentes_moto_sp (58161, 82)


In [30]:
df_acidentes_moto_sp.columns

Index(['ID', 'Data do Acidente', 'Dia do Acidente', 'Mês do Acidente',
       'Ano do Acidente', 'Ano/Mês do Acidente', 'Dia da Semana',
       'Hora do Acidente', 'Turno', 'Município', 'Região Administrativa',
       'Logradouro', 'Numero/KM', 'Jurisdição', 'Administração', 'Conservação',
       'Latitude', 'Longitude', 'Condições da pista', 'Condições Climáticas',
       'Iluminação', 'Mão de direção', 'Obras na pista', 'Relevo',
       'Superfície da via', 'Tipo de pavimento', 'Tipo de pista', 'Traçado',
       'Veículos Envolvidos - Bicicleta', 'Veículos Envolvidos - Caminhão',
       'Veículos Envolvidos - Automóvel', 'Veículos Envolvidos - Outros',
       'Veículos Envolvidos - Motocicleta', 'Veículos Envolvidos - Ônibus',
       'Veículos Envolvidos - ND', 'Veículos Envolvidos - Pedestre',
       'Pessoas Envolvidas - Grave', 'Pessoas Envolvidas - Ileso',
       'Pessoas Envolvidas - Leve', 'Pessoas Envolvidas - Fatal',
       'Pessoas Envolvidas - Feridos', 'Pessoas Envolvidas 

In [ ]:
# Selecionar colunas pra exportar


In [ ]:
# export
df_acidentes_moto_sp.to_csv('df_acidentes_moto_sp.csv')